In [25]:
import torch
import pandas as pd
from torch import nn
from torch import optim
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader

In [26]:
class CustomDataset(Dataset):
    def __init__(self, file_path):
        df = pd.read_csv(file_path)
        self.a = df.iloc[:, 0].values
        self.b = df.iloc[:, 1].values
        self.c = df.iloc[:, 2].values
        self.y = df.iloc[:, 3].values
        self.y = list(map(self.string_to_vector, self.y))
        self.length = len(df)

    def string_to_vector(self, value):
        data = {"acute triangle": 0, "right triangle": 1, "obtuse triangle": 2}
        return data.get(value, None)

    def __getitem__(self, index):
        x = torch.FloatTensor(sorted([self.a[index], self.b[index], self.c[index]]))
        y = torch.LongTensor(self.y)[index]
        return x, y

    def __len__(self):
        return self.length

In [27]:
class CustomModel(nn.Module):
    def __init__(self):
        super(CustomModel, self).__init__()

        self.layer = nn.Sequential(
            nn.Linear(3, 3)
        )

    def forward(self, x):
        x = self.layer(x)
        return x

In [28]:
train_dataset = CustomDataset("./multiclass_classification_data.csv")
train_dataloader = DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)

device = "cuda" if torch.cuda.is_available() else "cpu"
model = CustomModel().to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [29]:
for epoch in range(10000):
    cost = 0.0

    for x, y in train_dataloader:
        x = x.to(device)
        y = y.to(device)

        output = model(x)
        loss = criterion(output, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        cost += loss

    cost = cost / len(train_dataloader)

    if (epoch + 1) % 1000 == 0:
        print(f"Epoch : {epoch + 1:4d}, Cost : {cost:.3f}")

Epoch : 1000, Cost : 0.196
Epoch : 2000, Cost : 0.127
Epoch : 3000, Cost : 0.094
Epoch : 4000, Cost : 0.075
Epoch : 5000, Cost : 0.062
Epoch : 6000, Cost : 0.053
Epoch : 7000, Cost : 0.046
Epoch : 8000, Cost : 0.041
Epoch : 9000, Cost : 0.036
Epoch : 10000, Cost : 0.033


In [ ]:
with torch.no_grad():
    model.eval()
    classes = {0: "acute triangle", 1: "right triangle", 2: "obtuse triangle"}
    inputs = torch.FloatTensor([
        [9.02, 9.77, 9.96],  # 0 | acute triangle
        [8.01, 8.08, 8.32],  # 0 | acute triangle
        [3.55, 5.15, 6.26],  # 1 | right triangle
        [3.32, 3.93, 5.14],  # 1 | right triangle
        [4.39, 5.56, 9.99],  # 2 | obtuse triangle
        [3.01, 3.08, 9.98],  # 2 | obtuse triangle
        [5.21, 5.38, 5.39],  # 0 | acute triangle
        [3.85, 6.23, 7.32],  # 1 | right triangle
        [4.16, 4.98, 8.54],  # 2 | obtuse triangle
    ]).to(device)
    outputs = model(inputs)

    print('---------')
    print(outputs)
    print(torch.round(F.softmax(outputs, dim=1), decimals=2))
    print(outputs.argmax(1))
    print(list(map(classes.get, outputs.argmax(1).tolist())))